![The glorious SpatialIndex logo](img/spatial_index_logo.png)
# SpatialIndex Advanced Tutorial with Jupyter Notebook

##Introduction
If you're here and you haven't followed the previous tutorial `basic_tutorial.ipynb` maybe you should. It will give you the basics to understand what's going on here.

Or don't do it, I won't force you, I'm not your boss.

## Advanced usage
### Multi-indexing

If you have a huge circuit to index, you can resort to multi-indexing in order to index the big circuit faster. This techniques splits the whole circuit in multiple subtrees that can be processed in parallel using MPI. 

To do so you can create a multi-index object like this:

**Keep in mind that this and the following code won't work directly in this Jupyter notebook due to the necessity of MPI support. You can create a script with this code and run it using the usual `srun` for example.**

In [ ]:
import spatial_index
from spatial_index import MorphMultiIndexBuilder

OUTPUT_DIR = "multi_index_2k"

MorphMultiIndexBuilder.from_sonata_file(
    MORPH_FILE,
    NODE_FILE,
    "All",
    output_dir=OUTPUT_DIR,
)

We're specifying the same circuit files as before and then also supplying a folder that will contain all the subtrees that are generated. 

Once this process is complete, you can load the generated index using the `open_index` method of the `MorphMultiIndexBuilder` class, specifying the output folder of the previous execution and the amount of memory to allocate for loading the index.

You can then perform normal queries like on the usual index.

In [ ]:
# The index may use at most roughly 1e6 bytes.
index = spatial_index.open_index(OUTPUT_DIR, mem=int(1e6))

found = index.find_intersecting_window_np(min_corner, max_corner)
print(found)

### Memory Mapping

Alternatively, in case of a a big index and/or nodes with limited amount of memory, you can perform the indexing operation not in memory but directly on disk. This technique is known as Memory Mapping and allows to index huge circuit that would normally not fit inside the limits of the memory.

**On BB5 we recommend that you DON'T use GPFS for these purposes. Please allocate a node with access to fast NVME storage. Execution of memory mapped index building on GPFS will lead to long execution time and possibly disruption to the performance of GPFS for all the users of BB5.**

You can index a circuit with memory mapping directly from the command line (or better from a bash script) like this:

In [ ]:
%%bash

# Store the memory mapped index in this location 
TMP_FILE="/nvme/$USER/$SLURM_JOB_ID/segment_index.bin"

# Specify circuit files
NODES_FILE=/gpfs/bbp.cscs.ch/project/proj12/spatial_index/v0/circuit-2k/circuit.mvd3
MORPHOLOGY_LIB=/gpfs/bbp.cscs.ch/project/proj12/spatial_index/v0/circuit-2k/morphologies/ascii

# Run the CLI command using the --use-mem-map clause, specifying the max size of the index in MB
# The --shrink-on-close option allows the index to be automatically resized at the end of the execution
# possibly saving some space on disk.

spatial-index-nodes --use-mem-map=1000000 --shrink-on-close $NODES_FILE $MORPHOLOGY_LIB -o $TMP_FILE

# Copy the file elsewhere at the end of the execution
mv $TMP_FILE ./

This will create an index in the specified location. If, as you should, you're running your code in a allocated instance with access to NVME storage, be sure to copy the file elsewhere at the end of the execution or it might be lost once the allocation is over.

After the index is created, you can simply load it into any of your Python scripts using the `load_disk_mem_map` method of the `MorphIndexBuilder` class and then perform queries normally.

In [ ]:
index_pre = spatial_index.open_index("/path/to/memory_map/index")

This concludes the advanced tutorial. You now have all the tools, basics and advanced, to use SpatialIndex.

**Thanks a lot for your time and for making until the end of this tutorial. 🎉🎉🎉**

The SpatialIndex team hopes that this tool will make your life easier, lead to better results but mostly that you'll have fun with it as much as we had fun creating it!

**Disclaimer:** This tool is not meant to be fun but we have no right of deciding what's fun and what's not so, yeah, we just hope you don't hate it. And if you do, please let us know, kindly, and we'll do our best to improve it. Swearing is NEVER the answer.